# Performing a basic analysis <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **Compatability**: Notebook currently compatible with both the `NCI` and `DEA Sandbox` environments
* **Prerequisites**:  Users of this notebook should have a basic understanding of:
    * How to run a [Jupyter notebook](01_Jupyter_notebooks.ipynb)
    * The basic structure of the DEA [satellite datasets](02_DEA.ipynb)
    * Inspecting available [DEA products and measurements](03_Products_and_measurements.ipynb)
    * How to [load data from DEA](04_Loading_data.ipynb)
    * How to [plot loaded data](05_Plotting.ipynb)

## Background
To understand the world around us, it's important to combine the key steps of loading, visualising, manipulating and interpreting satellite data.
To perform an analysis, we begin with a question and use the steps above to reach an answer.

## Description
This notebook demonstrates how to conduct a basic analysis with DEA data and the Open Data Cube.
It will combine many of the steps that have been covered in the other beginner's notebooks.

In this notebook, the analysis question is "how is the health of vegetation changing over time for two different test locations?".
This could be related to a number of broader questions: 

* What is the effect of a particular fertilizer on a field of crops?
* How has a patch of forest changed after a fire? 
* How does proximity to water affect vegetation throughout the year?

For this notebook, the analysis question will be kept simple, without much real-world context. 
For more examples of notebooks that demonstrate how to use DEA to answer specific analysis questions, see the notebooks in the "Real world examples" folder. 

Topics covered in this notebook include:

1. Choosing a study area and two test locations
2. Exploring available data products for the area
3. Loading the chosen data
4. Plotting the chosen data and exploring how it changes with time
5. Calculating a measure of vegetation health from the loaded data
6. Generating a quantitative plot of how the vegetation health changes with time for the two test areas

***

## Getting started
To run this introduction to plotting data loaded from the datacube, run all the cells in the notebook starting with the "Load packages" cell. For help with running notebook cells, refer back to the [Jupyter Notebooks notebook](01_Jupyter_notebooks.ipynb). 

### Load packages
Import Python packages that are used for the analysis.

In [ ]:
%matplotlib inline

import datacube
import sys

sys.path.append("../Scripts")

### Connect to the datacube
The next step is to connect to the datacube database.
The resulting `dc` datacube object can then be used to load data.
The `app` parameter is a unique name used to identify the notebook that does not have any effect on the analysis.

In [ ]:
dc = datacube.Datacube(app="06_Basic_analysis")

## Step 1: Choose a study area and two test areas

### Study area
When working with the Open Data Cube, it's important to load only as much data as needed.
This helps keep an analysis running quickly and avoids the notebook crashing.

One way to set the study area is to set a central latitude and longitude coordinate pair, `(central_lat, central_lon)`, then specify how many degrees to include either side of the central latitude and longitude, known as the `buffer`.
Together, these parameters specify a square study area, as shown below:

<img src="../Supplementary_data/06_Basic_analysis/latlong_buffer.png" width="280" height="280">

Below, we have set the study area to be close to the town of Tully, Queensland.
The area contains both agriculture and forest, which will allow us to compare how the vegetation in these two areas has changed over time.

In [ ]:
central_lat = -17.942
central_lon = 145.887

buffer = 0.05

study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

print(study_area_lat)
print(study_area_lon)

### Test areas

Use markdown text for detailed, descriptive text explaining what the code below does and why it is needed.

In [ ]:
# Use code comments for low-level documentation of code
a = 1

### Subheading 1
Use subheadings to break up steps within a single section.

In [ ]:
# Use code comments for low-level documentation of code
b = 2

## Heading 2
Use markdown text for detailed, descriptive text explaining what the code below does and why it is needed.

In [ ]:
# Use code comments for low-level documentation of code
c = 3

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** September 2019

**Compatible `datacube` version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)